In [65]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from CryptoData import CryptoData

# Obtener los datos históricos
prices = CryptoData.get_historical_price('BTCUSDT', '1d')
prices['time'] = pd.to_datetime(prices['time'])

# Usar solo la columna 'close' para entrenamiento
data = prices[['open', 'high', 'low', 'close', 'volume']]

# Normalizar los datos
sc = MinMaxScaler(feature_range=(0, 1))
scaled_data = sc.fit_transform(data)

# Preparar los datos de entrenamiento
time_step = 30
x_train = []
y_train = []

for i in range(time_step, len(scaled_data)):
    x_train.append(scaled_data[i-time_step:i])
    y_train.append(scaled_data[i, 3]) # 3 corresponde a la columna 'close'
x_train, y_train = np.array(x_train), np.array(y_train)

# Reajustar los datos de entrenamiento para que sean 30*1
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2]))

# Crear y entrenar la red
dim_entrada = (x_train.shape[1], x_train.shape[2])
dim_salida = 1
na = 50

modelo = Sequential()
modelo.add(LSTM(units = na, input_shape=dim_entrada))
modelo.add(Dense(units = dim_salida))
modelo.compile(loss='mean_squared_error', optimizer='rmsprop')
modelo.fit(x_train, y_train, epochs=2000, batch_size=32)


Epoch 1/2000


d:\Descargas\Anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0168
Epoch 2/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0024
Epoch 3/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0022
Epoch 4/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0019
Epoch 5/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0020
Epoch 6/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0018
Epoch 7/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0017
Epoch 8/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0023
Epoch 9/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0013  
Epoch 10/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0011   
Epoch 11/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0013
Epoch 12/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0017
Epoch 13/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0013
Epoch 14/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0012
Epoch 15/2000
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.00

In [66]:
input_data = scaled_data[-time_step:]
input_data

array([[0.83225286, 0.83629181, 0.84689005, 0.82625946, 0.0209417 ],
       [0.82625961, 0.8338267 , 0.82871033, 0.83930476, 0.0356864 ],
       [0.83930491, 0.84262643, 0.78262988, 0.78355441, 0.07096173],
       [0.78355473, 0.77488827, 0.73538844, 0.74328587, 0.10547539],
       [0.743286  , 0.7537155 , 0.74181589, 0.75542807, 0.06126819],
       [0.75542802, 0.8182451 , 0.77582182, 0.8221295 , 0.0560614 ],
       [0.82212965, 0.83925029, 0.84259392, 0.83975929, 0.0307085 ],
       [0.83975926, 0.84109498, 0.84762711, 0.84185315, 0.02301834],
       [0.8418533 , 0.85595698, 0.84543993, 0.82707233, 0.0442753 ],
       [0.8270723 , 0.83720389, 0.83758061, 0.81218154, 0.03232777],
       [0.81218151, 0.81280185, 0.81300012, 0.79264884, 0.03301543],
       [0.79264898, 0.81991629, 0.80838121, 0.82548081, 0.03899125],
       [0.82548078, 0.82061415, 0.80045244, 0.78578844, 0.04671641],
       [0.78578858, 0.7866068 , 0.80582273, 0.78624227, 0.01623616],
       [0.7862424 , 0.79309805, 0.

In [67]:

# Número de días futuros que quieres predecir
futuro_dias = 30

# Obtener los últimos `time_step` datos de entrenamiento
input_data = scaled_data[-time_step:]

# Crear una lista para almacenar las predicciones futuras
futuro_predicciones = []

for i in range(futuro_dias):
    input_reshaped = np.reshape(input_data, (1, input_data.shape[0], input_data.shape[1]))
    prediccion = modelo.predict(input_reshaped)
    futuro_predicciones.append(prediccion[0, 0])
    
    # Crear un nuevo array con la última predicción y eliminar el primer elemento
    # Ajustar la predicción a la forma esperada
    nueva_fila = np.zeros(input_data.shape[1])
    nueva_fila[0] = prediccion[0, 0]
    new_data = np.append(input_data[1:], np.reshape(nueva_fila, (1, -1)), axis=0)
    input_data = new_data

# Inversa de la normalización de las predicciones futuras
futuro_predicciones = np.array(futuro_predicciones).reshape(-1, 1)
futuro_predicciones = sc.inverse_transform(np.concatenate((scaled_data[-futuro_dias:, :-1], futuro_predicciones), axis=1))[:, -1]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━

In [68]:
ult_fecha = prices['time'].iloc[-1]

futuras_fechas = pd.date_range(start=ult_fecha, periods=30)


In [69]:
import plotly.express as px
# Crear un rango de fechas futuras

# Crear un DataFrame con las predicciones futuras
df_futuro = pd.DataFrame({
    'Fecha': futuras_fechas,
    'Prediccion': futuro_predicciones
})

# Datos actuales con las predicciones futuras
df_resultado = pd.concat([prices[['time', 'close']].rename(columns={'time': 'Fecha', 'close': 'Valor Real'}), df_futuro], ignore_index=True)

df_resultado['Fecha'] = pd.to_datetime(df_resultado['Fecha'])
df_resultado['Prediccion'] = df_resultado['Prediccion'].astype(float)
df_resultado['Valor Real'] = df_resultado['Valor Real'].astype(float)

fig = px.line(df_resultado, x='Fecha', y=['Valor Real', 'Prediccion'], template='plotly_dark')
fig.show()


d:\Descargas\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [70]:
df_resultado['Valor Real'].head(1000)

0      48810.52
1      49246.64
2      49999.14
3      49915.64
4      51756.88
         ...   
995    67969.65
996    68549.99
997    69290.57
998    68507.67
999    69236.00
Name: Valor Real, Length: 1000, dtype: float64

In [71]:
df_resultado['Prediccion'][1011]


-1364.501900185364